ECE 8803 Final Project

In [22]:
import numpy as np
import pandas as pd
import os
from dataloader import OCTDataset
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image
import argparse
import os
import copy
import matplotlib.pyplot as plt



root = os.getcwd()
train = os.path.join(root, "df_prime_train.csv")
test = os.path.join(root, "df_prime_test.csv")



LABELS_SEVERITY = {35: 0,
                   43: 0,
                   47: 1,
                   53: 1,
                   61: 2,
                   65: 2,
                   71: 2,
                   85: 2}






'''Obtain Dataset Images'''


mean = (.1706)
std = (.2112)
normalize = transforms.Normalize(mean=mean, std=std)

transform = transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    normalize,
])

args = 'C:/Users/jgril/Documents/GitHub/8803_Final_Project'
image_trainset = OCTDataset(args, 'train', transform=transform)
image_testset = OCTDataset(args, 'test', transform=transform)


print("loading data")

images_train = []
labels_train = []
for i in range(len(image_trainset)):
    image,label = image_trainset.__getitem__(i)
    images_train.append(image.flatten())
    labels_train.append(label)

images_test = []
labels_test = []
for i in range(len(image_testset)):
    image,label = image_trainset.__getitem__(i)
    images_test.append(image.flatten())
    labels_test.append(label)

print("data loaded")

'''Meta Train Data'''
meta_trainset = pd.read_csv(train).dropna()
meta_testset = pd.read_csv(test).dropna()

train = os.path.join(root, "df_prime_train.csv")
test = os.path.join(root, "df_prime_test.csv")

meta_X_train = meta_trainset[['Age', 'Gender', 'Race', "Diabetes_Type", "Diabetes_Years", "BMI", "BCVA", "CST", "Leakage_Index"]].to_numpy()
meta_Y_train = meta_trainset[["DRSS"]].to_numpy()
y = []
for i in range(meta_Y_train.shape[0]):
    y.append(LABELS_SEVERITY[int(meta_Y_train[i])])

meta_Y_train = np.array(y)



'''Meta Test Data'''
meta_X_test = meta_testset[['Age', 'Gender', 'Race', "Diabetes_Type", "Diabetes_Years", "BMI", "BCVA", "CST", "Leakage_Index"]].to_numpy()
meta_Y_test = meta_testset[["DRSS"]].to_numpy() 
y = []
for i in range(meta_Y_test.shape[0]):
    y.append(LABELS_SEVERITY[int(meta_Y_test[i])])

Y_test = np.array(y)


#print(meta_X_train)
#print(meta_Y_train)
#print(meta_X_test)
#print(meta_Y_test)

loading data
data loaded


Naive Bayes Implementation

In [24]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

print(type(image_trainset[0]))
print(image_trainset[0])
X_train = np.numpy(images_train)
y_train = np.numpy(labels_train)

X_test = np.numpy(images_test)
y_test = np.numpy(labels_test)


# train a Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# evaluate the classifier on the test data
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

<class 'tuple'>
(tensor([[[-0.6592, -0.7149, -0.6778,  ..., -0.8078, -0.8078, -0.8078],
         [-0.1950, -0.4550, -0.3621,  ..., -0.8078, -0.8078, -0.8078],
         [-0.4735, -0.6221, -0.6407,  ..., -0.8078, -0.8078, -0.8078],
         ...,
         [-0.6221, -0.7706, -0.7892,  ..., -0.8078, -0.8078, -0.8078],
         [-0.7335, -0.7892, -0.7706,  ..., -0.8078, -0.8078, -0.8078],
         [-0.8078, -0.7892, -0.7706,  ..., -0.8078, -0.8078, -0.8078]]]), 2)


AttributeError: module 'numpy' has no attribute 'numpy'

SVM

In [16]:
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load iris dataset

X_train, X_test, y_train, y_test 
 = images_train.numpy()
y = labels_train

# Split dataset into training and test sets

# Create SVM classifier
clf = svm.SVC(kernel='linear', C=1, decision_function_shape='ovr')

# Train classifier
clf.fit(X_train, y_train)

# Predict test set
y_pred = clf.predict(X_test)

# Evaluate performance
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

KeyboardInterrupt: 